In [1]:
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
import tensorflow as tf
from tensorflow.contrib import learn
from data_utils import *
#from cnn import TextCNN
embedding_size = 128
filter_sizes =[3,4]
num_filters = 32
num_classes =20
batch_size = 64
num_epochs = 10

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


In [3]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


In [4]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

In [5]:
def max_pool(x, ksize):
    return tf.nn.max_pool(x, ksize,strides=[1, 1, 1, 1], padding='VALID')

In [6]:
x_train, y_train, x_test, y_test = load_data_and_labels()
#x_train=x_trai[0:1000]
#y_train=y_trai[0:1000]
        
print("length of training example for few",len(x_train))
print("length of training labels for few",len(y_train))
max_document_length = max([len(x.split(" ")) for x in x_train])
print("max length of doc",max_document_length)
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x_train = np.array(list(vocab_processor.fit_transform(x_train)))
x_test = np.array(list(vocab_processor.fit_transform(x_test)))
sequence_length=x_train.shape[1]
num_classes=y_train.shape[1]
vocab_size=len(vocab_processor.vocabulary_)
print("number of classes",num_classes)

length of training example for few 11314
length of training labels for few 11314
max length of doc 21273
number of classes 20


In [7]:
input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")

In [8]:
W = weight_variable([vocab_size, embedding_size])
embedded_chars = tf.nn.embedding_lookup(W,input_x)
embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)

In [9]:
pooled_outputs = []
for i, filter_size in enumerate(filter_sizes):
    # Convolution Layer
    filter_shape = [filter_size, embedding_size, 1, num_filters]
    W = weight_variable(filter_shape)
    b = bias_variable([num_filters])
    conv = conv2d(embedded_chars_expanded, W)
    # Apply nonlinearity
    h = tf.nn.relu(tf.nn.bias_add(conv, b))
    # Maxpooling over the outputs
    pooled = max_pool(h, [1, sequence_length - filter_size + 1, 1, 1])
    pooled_outputs.append(pooled)
print(len(pooled_outputs))    
num_filters_total = 32 * len(filter_sizes)
print(num_filters_total)
h_pool = tf.concat(pooled_outputs,2)
print(h_pool.shape)
h_pool_flat = tf.reshape(h_pool, [-1, num_filters_total])
W = weight_variable([num_filters_total, num_classes])
b = bias_variable([num_classes])
scores = tf.nn.softmax(tf.matmul(h_pool_flat, W) + b)
print(scores.shape)
pred = tf.nn.softmax(scores)
predictions = tf.argmax(scores, 1)

2
64
(?, 1, 2, 32)
(?, 20)


In [10]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                    logits=scores, labels=input_y))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
train_op = optimizer.minimize(loss_op)
correct_predictions = tf.equal(predictions, tf.argmax(input_y,1))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"))

In [11]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    print(data.shape)
    data_size = len(data)
    num_batches_per_epoch = int(len(data)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]


In [12]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [13]:
print(x_train.shape)
print(y_train.shape)
print(len(np.array(list(zip(x_train, y_train)))))
batches = batch_iter(list(zip(x_train, y_train)), batch_size, num_epochs)
for batch in batches:
    x_batch, y_batch = list(zip(*batch))
    _, cost = sess.run([train_op,loss_op], feed_dict={input_x: x_batch,input_y: y_batch})
    print( "Loss: {}".format(cost))
    test_accuracy = sess.run(accuracy,feed_dict={input_x: x_batch,input_y: y_batch})
    #print("test accuracy: {}".format(test_accuracy))

(11314, 21273)
(11314, 20)
11314
(11314, 2)
Loss: 2.9932801723480225
Loss: 2.995255470275879
Loss: 2.9988856315612793
Loss: 2.9962081909179688
Loss: 2.995871067047119
Loss: 2.9895544052124023
Loss: 2.9961633682250977
Loss: 2.9995310306549072
Loss: 2.9942736625671387
Loss: 2.9964005947113037
Loss: 2.990593433380127
Loss: 2.999326705932617
Loss: 2.9931840896606445
Loss: 2.9911985397338867
Loss: 2.995427131652832
Loss: 2.9969229698181152
Loss: 2.993962287902832
Loss: 2.989734172821045
Loss: 2.9917750358581543
Loss: 2.9932680130004883
Loss: 3.001214027404785
Loss: 2.9912376403808594
Loss: 2.9941539764404297
Loss: 2.9958415031433105
Loss: 2.9920949935913086
Loss: 2.989774227142334
Loss: 2.9956130981445312
Loss: 2.995276689529419
Loss: 2.9890284538269043
Loss: 2.9984796047210693
Loss: 2.9909555912017822
Loss: 2.990151882171631
Loss: 2.9920740127563477
Loss: 2.99271821975708
Loss: 2.99293851852417
Loss: 2.9907948970794678
Loss: 2.994096279144287
Loss: 2.9962825775146484
Loss: 2.99145412445068

Loss: 2.6276707649230957
Loss: 2.5285849571228027
Loss: 2.622931957244873
Loss: 2.612987518310547
Loss: 2.6378281116485596
Loss: 2.671562433242798
Loss: 2.659072160720825
Loss: 2.6656460762023926
Loss: 2.641119956970215
Loss: 2.6773128509521484
Loss: 2.658395767211914
Loss: 2.6429176330566406
Loss: 2.515225410461426
Loss: 2.6244542598724365
Loss: 2.555297613143921
Loss: 2.6732020378112793
Loss: 2.6799206733703613
Loss: 2.5840554237365723
Loss: 2.660921812057495
Loss: 2.542893409729004
Loss: 2.4767441749572754
Loss: 2.6388072967529297
Loss: 2.5800628662109375
Loss: 2.5411911010742188
Loss: 2.6165099143981934
Loss: 2.540222644805908
Loss: 2.575862407684326
Loss: 2.5564374923706055
Loss: 2.546225070953369
Loss: 2.572535514831543
Loss: 2.470294237136841
Loss: 2.5082149505615234
Loss: 2.57454776763916
Loss: 2.5707528591156006
Loss: 2.645537853240967
Loss: 2.501861572265625
Loss: 2.575679063796997
Loss: 2.5586509704589844
Loss: 2.5584452152252197
Loss: 2.490914821624756
Loss: 2.5599951744079

Loss: 2.3124542236328125
Loss: 2.2162041664123535
Loss: 2.353451728820801
Loss: 2.286012649536133
Loss: 2.32828950881958
Loss: 2.2670934200286865
Loss: 2.3116707801818848
Loss: 2.309291124343872
Loss: 2.328505039215088
Loss: 2.278268814086914
Loss: 2.2525854110717773
Loss: 2.275333881378174
Loss: 2.3100814819335938
Loss: 2.2996444702148438
Loss: 2.354173421859741
Loss: 2.2831077575683594
Loss: 2.3161709308624268
Loss: 2.293020248413086
Loss: 2.3363406658172607
Loss: 2.2812561988830566
Loss: 2.307560443878174
Loss: 2.3157012462615967
Loss: 2.2569470405578613
Loss: 2.303253650665283
Loss: 2.230196952819824
Loss: 2.2931013107299805
Loss: 2.2403550148010254
Loss: 2.250673770904541
Loss: 2.319026470184326
Loss: 2.3043861389160156
Loss: 2.2663683891296387
Loss: 2.245804786682129
Loss: 2.322495937347412
Loss: 2.2858872413635254
Loss: 2.3568577766418457
Loss: 2.2878496646881104
Loss: 2.2512478828430176
Loss: 2.2459731101989746
Loss: 2.274606704711914
Loss: 2.204158067703247
Loss: 2.32385873794

Loss: 2.1814393997192383
Loss: 2.2104382514953613
Loss: 2.1716177463531494
Loss: 2.1148204803466797
Loss: 2.1787030696868896
Loss: 2.192112922668457
Loss: 2.1681885719299316
Loss: 2.1879916191101074
Loss: 2.2115073204040527
Loss: 2.174830913543701
Loss: 2.148698329925537
Loss: 2.1949462890625
Loss: 2.179013967514038
Loss: 2.174501895904541
Loss: 2.1277108192443848
Loss: 2.2325587272644043
Loss: 2.167574405670166
Loss: 2.201584815979004
Loss: 2.168069839477539
Loss: 2.2016844749450684
Loss: 2.125929355621338
Loss: 2.1760261058807373
Loss: 2.199984550476074
Loss: 2.1852030754089355
Loss: 2.166743040084839
Loss: 2.2013323307037354
Loss: 2.1631596088409424
Loss: 2.1388673782348633
Loss: 2.1233088970184326
Loss: 2.2328882217407227
Loss: 2.174056053161621
Loss: 2.128298044204712
Loss: 2.118079662322998
Loss: 2.2042081356048584
Loss: 2.165684223175049
Loss: 2.161465883255005
Loss: 2.2057247161865234
Loss: 2.1711509227752686
Loss: 2.193124771118164
Loss: 2.14871883392334
Loss: 2.14330625534057

Loss: 2.1719560623168945
Loss: 2.148836612701416
Loss: 2.2010836601257324
Loss: 2.110286235809326
Loss: 2.1112184524536133
Loss: 2.18599796295166
Loss: 2.1404175758361816
Loss: 2.201394557952881
Loss: 2.1565284729003906
Loss: 2.1400437355041504
Loss: 2.172105550765991
Loss: 2.187129020690918
Loss: 2.140197515487671
Loss: 2.1558632850646973
Loss: 2.1865234375
Loss: 2.1713733673095703
Loss: 2.1716196537017822
Loss: 2.1250298023223877
Loss: 2.1104698181152344
Loss: 2.1572818756103516
Loss: 2.17134952545166
Loss: 2.140850067138672
Loss: 2.2170372009277344
Loss: 2.1590065956115723
Loss: 2.2013742923736572
Loss: 2.145000457763672
Loss: 2.1259851455688477
Loss: 2.186431646347046
Loss: 2.2159955501556396
Loss: 2.1403589248657227
Loss: 2.15541672706604
Loss: 2.2479944229125977
Loss: 2.1400797367095947
Loss: 2.1414294242858887
Loss: 2.09542179107666
Loss: 2.1405510902404785
Loss: 2.125600576400757
Loss: 2.2012367248535156
Loss: 2.1866559982299805
Loss: 2.1573426723480225
Loss: 2.125652551651001


Loss: 2.1425440311431885
Loss: 2.12508487701416
Loss: 2.125429153442383
Loss: 2.1409106254577637
Loss: 2.09523868560791
Loss: 2.0798206329345703
Loss: 2.080411911010742
Loss: 2.110106945037842
Loss: 2.127182960510254
Loss: 2.1103262901306152
Loss: 2.0950188636779785
Loss: 2.155172824859619
Loss: 2.096759080886841
Loss: 2.080007553100586
Loss: 2.126373291015625
Loss: 2.12546968460083
Loss: 2.110893487930298
Loss: 2.1258935928344727
Loss: 2.1259286403656006
Loss: 2.1554458141326904
Loss: 2.0791947841644287
Loss: 2.12530517578125
Loss: 2.110015869140625
Loss: 2.126112461090088
Loss: 2.0814876556396484
Loss: 2.109929084777832
Loss: 2.118425130844116
Loss: 2.0956928730010986
Loss: 2.169429063796997
Loss: 2.1416478157043457
Loss: 2.0952506065368652
Loss: 2.09515380859375
Loss: 2.1598525047302246
Loss: 2.140629291534424
Loss: 2.1259994506835938
Loss: 2.079000473022461
Loss: 2.0949554443359375
Loss: 2.091393232345581
Loss: 2.1105098724365234
Loss: 2.1567864418029785
Loss: 2.0973153114318848
Lo

In [14]:
print(x_test.shape)
print(y_test.shape)
sum1=0
num1=0
print(len(np.array(list(zip(x_test, y_test)))))
batches = batch_iter(list(zip(x_test, y_test)), batch_size, 1)
for batch in batches:
    num1=num1+1
    x_batch, y_batch = list(zip(*batch))
    test_accuracy = sess.run(accuracy,feed_dict={input_x: x_batch,input_y: y_batch})
    print("test accuracy: {}".format(test_accuracy))
    #acc=np.mean(idx==true_idx)
    #print(acc)
    sum1=sum1+test_accuracy

(7532, 21273)
(7532, 20)
7532
(7532, 2)
test accuracy: 0.75
test accuracy: 0.734375
test accuracy: 0.71875
test accuracy: 0.828125
test accuracy: 0.765625
test accuracy: 0.78125
test accuracy: 0.8125
test accuracy: 0.75
test accuracy: 0.734375
test accuracy: 0.75
test accuracy: 0.75
test accuracy: 0.734375
test accuracy: 0.75
test accuracy: 0.78125
test accuracy: 0.78125
test accuracy: 0.71875
test accuracy: 0.828125
test accuracy: 0.828125
test accuracy: 0.828125
test accuracy: 0.734375
test accuracy: 0.765625
test accuracy: 0.8125
test accuracy: 0.8125
test accuracy: 0.796875
test accuracy: 0.765625
test accuracy: 0.75
test accuracy: 0.859375
test accuracy: 0.8125
test accuracy: 0.796875
test accuracy: 0.828125
test accuracy: 0.765625
test accuracy: 0.765625
test accuracy: 0.71875
test accuracy: 0.71875
test accuracy: 0.765625
test accuracy: 0.765625
test accuracy: 0.75
test accuracy: 0.734375
test accuracy: 0.71875
test accuracy: 0.78125
test accuracy: 0.78125
test accuracy: 0.6875


In [15]:
print(num1)
print(sum1)
total1=sum1*100/num1
print(total1)

118
90.1576704979
76.4048055067


In [16]:
print(x_train.shape)
print(y_train.shape)
sum2=0
num2=0
print(len(np.array(list(zip(x_train, y_train)))))
batches = batch_iter(list(zip(x_train, y_train)), batch_size, 1)
for batch in batches:
    num2=num2+1
    x_batch, y_batch = list(zip(*batch))
    train_accuracy = sess.run(accuracy,feed_dict={input_x: x_batch,input_y: y_batch})
    print("train accuracy: {}".format(train_accuracy))
    #acc=np.mean(idx==true_idx)
    #print(acc)
    sum2=sum2+train_accuracy

(11314, 21273)
(11314, 20)
11314
(11314, 2)
train accuracy: 1.0
train accuracy: 0.9375
train accuracy: 0.984375
train accuracy: 0.921875
train accuracy: 0.96875
train accuracy: 0.96875
train accuracy: 1.0
train accuracy: 0.96875
train accuracy: 0.9375
train accuracy: 0.96875
train accuracy: 0.9375
train accuracy: 0.921875
train accuracy: 0.9375
train accuracy: 0.96875
train accuracy: 0.96875
train accuracy: 0.984375
train accuracy: 0.9375
train accuracy: 0.984375
train accuracy: 0.953125
train accuracy: 0.984375
train accuracy: 0.953125
train accuracy: 0.90625
train accuracy: 1.0
train accuracy: 1.0
train accuracy: 0.9375
train accuracy: 0.9375
train accuracy: 0.953125
train accuracy: 0.953125
train accuracy: 0.984375
train accuracy: 0.984375
train accuracy: 0.984375
train accuracy: 0.984375
train accuracy: 0.90625
train accuracy: 0.953125
train accuracy: 0.953125
train accuracy: 0.96875
train accuracy: 0.953125
train accuracy: 0.96875
train accuracy: 0.984375
train accuracy: 1.0
train

In [17]:
print(num2)
print(sum2)
total2=sum2*100/num2
print(total2)

177
170.625
96.3983050847
